In [ ]:
# Sampling training data
from google.cloud import bigquery
client = bigquery.Client()

sql = """
SELECT year, plurality, apgar_5min, 
       mother_age, father_age,    
       gestation_weeks, ever_born
       ,case when mother_married = true 
             then 1 else 0 end as mother_married
       ,weight_pounds as weight
  FROM  `bigquery-public-data.samples.natality`
  order by rand() 
  limit 10000
"""

natalityDF = client.query(sql).to_dataframe().fillna(0)
natalityDF.head()

In [ ]:
# Fitting a model and saving to cloud
from sklearn.linear_model import LinearRegression
import pickle
from google.cloud import storage

# fit and pickle a model 
model = LinearRegression()
model.fit(natalityDF.iloc[:,1:8], natalityDF['weight'])
pickle.dump(model, open("natality.pkl", 'wb'))

# Save to GCS
bucket = storage.Client().get_bucket('dsp_model_store')
blob = bucket.blob('natality/sklearn-linear')
blob.upload_from_filename('natality.pkl')

In [ ]:
# Real data (without limit)
import apache_beam as beam
import argparse
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.io.gcp.bigquery import parse_table_schema_from_json
import json

query = """
    SELECT year, plurality, apgar_5min, 
    mother_age, father_age,    
       gestation_weeks, ever_born
       ,case when mother_married = true 
          then 1 else 0 end as mother_married
      ,weight_pounds as weight
      ,current_timestamp as time
      ,GENERATE_UUID() as guid
    FROM `bigquery-public-data.samples.natality` 
    rand() 
"""

In [ ]:
class ApplyDoFn(beam.DoFn):

    def __init__(self):
        self._model = None
        from google.cloud import storage
        import pandas as pd
        import pickle as pkl
        self._storage = storage
        self._pkl = pkl
        self._pd = pd
     
    def process(self, element):
        if self._model is None:
            bucket = self._storage.Client().get_bucket('model_store')
            blob = bucket.get_blob('natality/sklearn-linear')
            self._model =self._pkl.loads(blob.download_as_string())
        
        new_x = self._pd.DataFrame.from_dict(element, 
                            orient = "index").transpose().fillna(0)   
        weight = self._model.predict(new_x.iloc[:,1:8])[0]
        return [ { 'guid': element['guid'], 'weight': weight, 
                                   'time': str(element['time']) } ]

In [ ]:
schema = parse_table_schema_from_json(json.dumps({'fields':
            [ { 'name': 'guid', 'type': 'STRING'},
              { 'name': 'weight', 'type': 'FLOAT64'},
              { 'name': 'time', 'type': 'STRING'} ]}))

In [ ]:
# set up pipeline options 
parser = argparse.ArgumentParser()
known_args, pipeline_args = parser.parse_known_args(None)
pipeline_options = PipelineOptions(pipeline_args)

# define the pipeline steps
p = beam.Pipeline(options=pipeline_options)
data = p | 'Read from BigQuery' >> beam.io.Read(
       beam.io.BigQuerySource(query=query, use_standard_sql=True))
scored = data | 'Apply Model' >> beam.ParDo(ApplyDoFn())
scored | 'Save to BigQuery' >> beam.io.Write(beam.io.BigQuerySink(
                'weight_preds', 'dataflow_beam_pipeline', schema = schema,
   create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
   write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND))

# run the pipeline
result = p.run()
result.wait_until_finish()

In [ ]:
# Publish to datastore
class PublishDoFn(beam.DoFn):
    
    def __init__(self):
        from google.cloud import datastore       
        self._ds = datastore
    
    def process(self, element):
        client = self._ds.Client()
        key = client.key('natality-guid', element['guid'])
        entity = self._ds.Entity(key)
        entity['weight'] = element['weight']         
        entity['time'] = element['time']
        client.put(entity)

scored | 'Create entities' >> beam.ParDo(PublishDoFn())

# Testing (datastore publish)
from google.cloud import datastore
client = datastore.Client()
query = client.query(kind='natality-guid')

query_iter = query.fetch()
for entity in query_iter:
    print(entity)
    break